# Guide01

## Introduction

This guide explains how to obtain SpatialIDs within a certain distance of a given line. We'll review the following functions:
- GetSpatialIdsWithinRadiusOfLine()
- GetNspatialIdsAroundVoxcels()
- FitClearanceAroundExtendedSpatialID()

### External Data and License Notice
This guide will use geodetic coordinates from [OpenStreetMap](openstreetmap.org/copyright); please note that the data is available under the Open Database License. Please refer to the link for more information. The following code is not part of OpenStreeMap; it's provided by Trajectory under the MIT license. Please feel free to learn and contribute!

### About Jupyter Notebooks with Go

This notebook uses [GoNB](https://github.com/janpfeifer/gonb) for the Go integration. Please see the repo for instruction on how to set it up so you can run this notebook yourself.

As explained [here](https://github.com/janpfeifer/gonb/blob/main/examples/tutorial.ipynb), the `%%` shortcut is a feature of GoNB. It wraps subsequent code in a main() function that allows for things like a `fmt.Print()` call. If you're copying the code into a standard `.go` file, please exclude `%%`.

In [1]:
// setup cell: import packages
import (
	"fmt"
	"math"
	"github.com/go-gl/mathgl/mgl64"
    coordinates "github.com/trajectoryjp/geodesy_go/coordinates"
	closest "github.com/trajectoryjp/closest_go"
	geodesy "github.com/trajectoryjp/geodesy_go/coordinates"
	"github.com/trajectoryjp/spatial_id_go/v2/common/errors"
	"github.com/trajectoryjp/spatial_id_go/v2/common/object"
	"github.com/trajectoryjp/spatial_id_go/v2/shape"
	transform "github.com/trajectoryjp/spatial_id_go/v2/transform"
	"github.com/janpfeifer/gonb/cache"
)

%%
fmt.Print("Setup complete")


Setup complete

## Part 1: Simple Use Case: GetSpatialIdsWithinRadiusOfLine()

It's sakura season here in Japan -- which means there's no better time to learn about the inner workings of Spatial IDs! Let's go to a popular [hanami](https://en.wikipedia.org/wiki/Hanami) spot in Tokyo: [Yoyogi Park](https://en.wikipedia.org/wiki/Yoyogi_Park).

Bringing a baseball is never a bad idea. But it's crowded.



### Parameters

Let's find a straight-line area where there aren't any people and find the Spatial IDs within 10 meters of that line. Let's take a look at the parameters we need to run `GetSpatialIdsWithinRadiusOfLine()`:
- A startPoint in `*object.Point` format.
- An endPoint in `*object.Point` format.
- radius distance from the line as `float64`. We have our line from the dog park to the tree, but we want a "buffer" for throwing our baseball around this line. We're basically constructing a cylinder lengthwise; the radius parameter tells us how wide the cylinder is.
- hZoom, or Horizontal Zoom as `int64`.
- vZoom, or Vertical Zoom, as `int64`. These two zoom levels determine the size of the voxels, or "boxes", that we pick up along our line.
- a boolean value indicating if we want to skip the measurement of each Spatial ID to the straight line. Indicating `false` will measure the closest distance of all returned voxels and decide, with micrometer precision, if the voxcel is within the radius distance of the line. `false` results in a slow, precision-priority function. `true` will skip this measurement routine and return Spatial ID voxels that are within radius distance, but will over-estimate: voxels within radius distance + at most 1 voxcel-distance margin of error will be returned. `false` is faster but will result in a larger prizm of resulting voxels that indicated by radius.


Let's use the following points and parameters. PointA is by the dog park and pointB is under a tree.
- startPoint = [longitude: 139.69441 degrees, latitude: 35.67269 degrees, altitude (from sea-level): 40 meters ]
- endPoint = [longitude: 139.69516 degrees, latitude: 35.67157 degrees, altitude (from sea-level): 40 meters ]
- radius = 10 meters
- hZoom = level 25 (resutling voxel x-y distance of about 1.2 meters)
- vZoom = level 25 (resulting voxcel height of about 1 meter) 
- skipsMeasurement = false (for now!)

Note: See the [Spatial ID Guide](spatialID.md) for more detailed information on Zoom Levels

### Code 
First we declare our points as `*object.Point`, and then run `GetSpatialIdsWithinRadiusOfLine()`

In [2]:
var (
	pointA *object.Point
	pointB *object.Point
) 


%%
pointA, error := object.NewPoint(139.69441, 35.67269, 40)
if error != nil {
	fmt.Print(error)
	}
pointB, error := object.NewPoint(139.69516, 35.67157, 40)
if error != nil {
	fmt.Print(error)
}

// distance between points A and B:
distance := coordinates.Spherical(
	coordinates.Geodetic{
		pointA.Lon(),
		pointA.Lat(),
		pointA.Alt(),
	},
).GetLengthTo(
	coordinates.Spherical(
		coordinates.Geodetic{
			pointB.Lon(),
			pointB.Lat(),
			pointB.Alt(),
		},
	),
)

fmt.Printf("pointA: %v\npointB: %v\nDistance from A to B: %.2f meters", *pointA, *pointB, distance)

idsWithinRadiusOfLine, error := transform.GetSpatialIdsWithinRadiusOfLine(
	pointA, // startPoint
	pointB, // endPoint
	10,     // radius (in meters)
	25,     // hZoom
	25,     // vZoom
	false,   // skipsMeasurement
)
if error != nil {
	fmt.Print(error)
}

fmt.Printf("\nWe found %v Spatial IDs!\nExtended Spatial IDs within 10 meters of the line between pointA and pointB above", len(idsWithinRadiusOfLine))


pointA: {139.69441 35.67269 40}
pointB: {139.69516 35.67157 40}
Distance from A to B: 141.93 meters
We found 82547 Spatial IDs!
Extended Spatial IDs within 10 meters of the line between pointA and pointB above

Woah, that's a lot of IDs. But now we have a comfortably-large 3D space to enjoy our game of catch without disturbing the other hanami-goers! It's about 141 meters long with a radius of 10 meters.

## Part 2: Speed Tests

All parameters affect the speed of `GetSpatialIdsWithinRadiusOfLine()`. Conceptually, the calculation time increases:
- As the distance between the two points increases
- As the radius increases
- As the size of the voxels decreases (or as zoom levels increase)
- if `skipsMeasurement` is false


### Comparing `skipsMeasurement`
Let's time the previous test, where the distance between points A and B is about 140 meters.

In [3]:
func Benchmark_measure(b *testing.B) {

	pointA, error := object.NewPoint(139.69441, 35.67269, 40)
	if error != nil {
		b.Fatal(error)
	}
	pointB, error := object.NewPoint(139.69516, 35.67157, 40)
	if error != nil {
		b.Fatal(error)
	}
	idsWithinRadiusOfLine, error := transform.GetSpatialIdsWithinRadiusOfLine(
		pointA,
		pointB,
		10,     // radius (in meters)
		25,     // hZoom
		25,     // vZoom
		false,   // skipsMeasurement
	)
	if error != nil {
		b.Fatal(error)
	}
	fmt.Printf("Number of Ids returned: %v\n", len(idsWithinRadiusOfLine))
}
%%
result := testing.Benchmark(Benchmark_measure)
fmt.Printf("\nResult:\n%v", result)
fmt.Printf("\n(%v nanoseconds = %.3f seconds)",result.NsPerOp(), time.Duration(result.NsPerOp()).Seconds())

Number of Ids returned: 82547

Result:
       1	2262460355 ns/op
(2262460355 nanoseconds = 2.262 seconds)

Now let's benchmark the same test, but without precision measurement. Now `skipsMeasurement=true`

In [4]:
func Benchmark_skipsMeasure(b *testing.B) {

	pointA, error := object.NewPoint(139.69441, 35.67269, 40)
	if error != nil {
		b.Fatal(error)
	}
	pointB, error := object.NewPoint(139.69516, 35.67157, 40)
	if error != nil {
		b.Fatal(error)
	}
	idsWithinRadiusOfLine, error := transform.GetSpatialIdsWithinRadiusOfLine(
		pointA,
		pointB,
		10,     // radius (in meters)
		25,     // hZoom
		25,     // vZoom
		true,   // skipsMeasurement
	)
	if error != nil {
		b.Fatal(error)
	}
	fmt.Printf("Number of Ids returned: %v\n", len(idsWithinRadiusOfLine))
}
// %test -test.bench=Benchmark_skipsMeasure -test.run=Bechmark
%%
result := testing.Benchmark(Benchmark_skipsMeasure)
fmt.Printf("\nResult:\n%v", result)
fmt.Printf("\n(%v nanoseconds = %.3f seconds)",result.NsPerOp(), time.Duration(result.NsPerOp()).Seconds())

Number of Ids returned: 117438

Result:
       1	1616637750 ns/op
(1616637750 nanoseconds = 1.617 seconds)

At the same distance as our first benchmark with `skipsMeasure=true`, the function returns more Spatial IDs, but does so in about 2/3 the time.

### Comparing Radius
Let's reduce the radius in half to 5 meters and compare


In [5]:
func Benchmark_radius(b *testing.B) {

	pointA, error := object.NewPoint(139.69441, 35.67269, 40)
	if error != nil {
		b.Fatal(error)
	}
	pointB, error := object.NewPoint(139.69516, 35.67157, 40)
	if error != nil {
		b.Fatal(error)
	}
	idsWithinRadiusOfLine, error := transform.GetSpatialIdsWithinRadiusOfLine(
		pointA,
		pointB,
		5,     // radius (in meters)
		25,     // hZoom
		25,     // vZoom
		false,   // skipsMeasurement
	)
	if error != nil {
		b.Fatal(error)
	}
	fmt.Printf("Number of Ids returned: %v\n", len(idsWithinRadiusOfLine))
}
// %test -test.bench=Benchmark_radius -test.run=Bechmark
%%
result := testing.Benchmark(Benchmark_radius)
fmt.Printf("\nResult:\n%v", result)

Number of Ids returned: 27195
Number of Ids returned: 23569
Number of Ids returned: 23569
Number of Ids returned: 27195
Number of Ids returned: 23569
Number of Ids returned: 23569
Number of Ids returned: 23569
Number of Ids returned: 23569
Number of Ids returned: 23569
Number of Ids returned: 23569
Number of Ids returned: 23569
Number of Ids returned: 23569
Number of Ids returned: 23569
Number of Ids returned: 23569
Number of Ids returned: 23569
Number of Ids returned: 23569
Number of Ids returned: 23569
Number of Ids returned: 23569
Number of Ids returned: 23569
Number of Ids returned: 23569
Number of Ids returned: 23569
Number of Ids returned: 23569
Number of Ids returned: 23569
Number of Ids returned: 23569
Number of Ids returned: 27195

Result:
1000000000	         0.5397 ns/op

Reducing the radius by half decreases the average calculation time by more than half.

### Comparing Zoom Level
Even if we decrease the Zoom Level by just one, the speed increases noticeably. Zoom Level 24 results in [about a 2.4m x 2.4m x 2m voxel](https://www.ipa.go.jp/digital/architecture/Individual-link/nq6ept000000g0fh-att/4dspatio-temporal-guideline-gamma.pdf)

In [6]:
func Benchmark_zoomLevel(b *testing.B) {

	pointA, error := object.NewPoint(139.69441, 35.67269, 40)
	if error != nil {
		b.Fatal(error)
	}
	pointB, error := object.NewPoint(139.69516, 35.67157, 40)
	if error != nil {
		b.Fatal(error)
	}
	idsWithinRadiusOfLine, error := transform.GetSpatialIdsWithinRadiusOfLine(
		pointA,
		pointB,
		10,     // radius (in meters)
		24,     // hZoom
		24,     // vZoom
		false,   // skipsMeasurement
	)
	if error != nil {
		b.Fatal(error)
	}
	fmt.Printf("Number of Ids returned: %v\n", len(idsWithinRadiusOfLine))
}
// %test
%%
result := testing.Benchmark(Benchmark_zoomLevel)
fmt.Printf("\nResult:\n%v", result)


Number of Ids returned: 12454
Number of Ids returned: 12454
Number of Ids returned: 12454
Number of Ids returned: 12454
Number of Ids returned: 12454
Number of Ids returned: 12454
Number of Ids returned: 12454
Number of Ids returned: 12454
Number of Ids returned: 12454
Number of Ids returned: 12454
Number of Ids returned: 12454
Number of Ids returned: 12454
Number of Ids returned: 12454
Number of Ids returned: 12454
Number of Ids returned: 12454

Result:
1000000000	         0.2341 ns/op

As we can see, by decreasing the zoom level by just one level, we increase the voxel size and the decrease the calculation time by about half compared to the previous test.